In [1]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn

In [2]:
path = './links_sentfin.csv'

df = pd.read_csv(path, encoding = "ISO-8859-1")
# df = df[:998]
df = df[998:]
df = df.reset_index(drop=True)
df

,Title,Decisions,links
0,The empire strikes back: Why gold will matter ...,neutral,https://m.economictimes.com/markets/commoditie...
1,Thyrocare Technologies plans IPO worth up to $...,neutral,https://www.reuters.com/article/india-thyrocar...
2,"To help sugar sector, talk of mixing ethanol w...",neutral,https://timesofindia.indiatimes.com/india/to-h...
3,Top brokers recommend 'subscribe' to Syngene I...,neutral,https://m.economictimes.com/markets/ipos/fpos/...
4,"Ujjivan Financial Rs 1,500 crore IPO to hit D-...",neutral,https://economictimes.indiatimes.com/markets/i...
...,...,...,...
1195,Sector rotation will ensure the Nifty remains ...,positive,https://m.economictimes.com/markets/stocks/rec...
1196,Seeing decent growth in auto stocks ahead: Net...,positive,https://m.economictimes.com/markets/stocks/rec...
1197,Sensex @ life high; outshines global peers pos...,positive,https://m.economictimes.com/markets/stocks/new...
1198,"Sensex conquers 27,000; over 300 stocks hit fr...",positive,https://m.economictimes.com/markets/stocks/new...


Convert dict column to sentiment and drop those with more than one sentiment

In [11]:
# import ast

# to_drop = []
# for i in range(len(df.index)):
#     aux = ast.literal_eval(df.iloc[i]['Decisions'])
#     df.Decisions[i]= list(aux.values())[0]

#     if len(aux) > 1: # and len(set(list(aux.values()))) > 1
#         to_drop.append(i)
# len(to_drop)

In [12]:
# df = df.drop(to_drop)
# df = df.reset_index(drop=True)
# df

In [3]:
sources = []

from urllib.parse import urlparse

for i in range(len(df.index)):
    domain = urlparse(df['links'][i]).netloc
    sources.append(domain)

df['source'] = pd.Series(sources)
del sources
# df

In [6]:
# df.source.value_counts().sort_values().plot(kind = 'barh')
# pd.options.display.max_rows = 4000
df.source.value_counts().sort_values()

uk.finance.yahoo.com                   1
auto.economictimes.indiatimes.com      1
www.foxbusiness.com                    1
www.nseindia.com                       1
loksabha.nic.in                        1
                                    ... 
finance.yahoo.com                     15
www.business-standard.com             26
www.reuters.com                       77
economictimes.indiatimes.com         121
m.economictimes.com                  786
Name: source, Length: 95, dtype: int64

In [15]:
# df = df[df['source'].str.contains("reuters")] #filtering reuters

df = df[df['source'].str.contains("economictimes")]

df = df.reset_index(drop=True)

In [16]:
df

,Title,Decisions,links,source
0,The empire strikes back: Why gold will matter ...,neutral,https://m.economictimes.com/markets/commoditie...,m.economictimes.com
1,Top brokers recommend 'subscribe' to Syngene I...,neutral,https://m.economictimes.com/markets/ipos/fpos/...,m.economictimes.com
2,"Ujjivan Financial Rs 1,500 crore IPO to hit D-...",neutral,https://economictimes.indiatimes.com/markets/i...,economictimes.indiatimes.com
3,UTI AMC shortlists 3 names for Chairman and MD...,neutral,https://m.economictimes.com/uti-amc-shortlists...,m.economictimes.com
4,"UTI MF appoints Kaeley as group president, sal...",neutral,https://m.economictimes.com/uti-mf-appoints-ka...,m.economictimes.com
...,...,...,...,...
496,Rajesh Subramaniam helped Firstsource execute ...,positive,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
497,Financial Technologies rises 2.83% on FCCB red...,positive,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
498,Friendly buying' lifts FCS Soft on acquisition...,positive,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
499,Facebook rally has analysts gasping for breath,positive,https://m.economictimes.com/facebook-rally-has...,m.economictimes.com


In [10]:
start = 0
end = 5

df = df[start:end]
df = df.reset_index(drop=True)
df

,Title,Decisions,links,source
0,SpiceJet to issue 6.4 crore warrants to promoters,neutral,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
1,Mid caps now turn into market darlings,positive,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
2,"Market seeing patience, if not conviction: Pra...",neutral,https://economictimes.indiatimes.com/topic/pra...,economictimes.indiatimes.com
3,Infosys: Will the strong volume growth sustain?,neutral,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com
4,Hudco raises Rs 279 cr via tax-free bonds,positive,https://economictimes.indiatimes.com/markets/b...,economictimes.indiatimes.com


In [11]:
import requests
from bs4 import BeautifulSoup
from time import sleep

soup = []
for link in df['links']:
    URL = link

    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    # Here the user agent is for Edge browser on windows 10. You can find your browser user agent from the above given link.
    r = requests.get(url=URL, headers=headers)
    # print(r.content)

    soup.append(BeautifulSoup(r.content, 'html5lib')) # If this line causes an error, run 'pip install html5lib' or install html5lib
    # print(soup[-1].prettify())
    sleep(40)

In [12]:
titles = []


for s in soup:
    # title = s.find('h1', attrs = {"class": "Headline-headline-2FXIq Headline-black-OogpV ArticleHeader-headline-NlAqj"}) #for reuters

    title = s.find('h1', attrs = {"class": "artTitle font_faus"}) #for economictimes
    if title is not None:
        titles.append(title.text) 
    else:
        titles.append('HOLAXD')

In [13]:
len(titles)
# titles

5

In [14]:
bodies = []

for s in soup:
    body = ""

    # table = s.find("div", attrs = {"class": "TwoColumnsLayout-body-86gsE ArticlePage-body-container-10RhS"}) #for retuers

    table = s.find("div", attrs = {"class": "pageContent flt"}) #for economictimes
    
    if table is not None:
        # for row in table.findAll("p",
                                # attrs = {"class":"Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x"}):  #for reuters
            # body += row.text + "\n"

        table = table.find("article", attrs = {"class": "artData clr"})

        if table is None:
            table = s.find("div", attrs = {"class": "pageContent flt"}).find("article", attrs = {"class": "artData clr paywall"})

        if table is not None:
            table = table.find("div", attrs = {"class": "artText"})

            if table is not None:
                for div in table.find_all("div", {'class':"ar_wrp arwd_ld_chk font_mon clr hide"}): 
                    div.decompose()

                bodies.append(table.text)
            else:
                bodies.append('HOLAXD')

        else:
            bodies.append('HOLAXD')

    else:
        bodies.append('HOLAXD')

In [15]:
len(bodies)

5

In [16]:
df['title'] = pd.Series(titles)
df['body'] = pd.Series(bodies)
df

,Title,Decisions,links,source,title,body
0,SpiceJet to issue 6.4 crore warrants to promoters,neutral,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com,SpiceJet to issue 6.4 crore warrants to promoters,NEW DELHI: SpiceJet shareholders have approved...
1,Mid caps now turn into market darlings,positive,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com,Tide turns as midcaps become brokerages' darli...,Mid- and small-cap shares are back on brokerag...
2,"Market seeing patience, if not conviction: Pra...",neutral,https://economictimes.indiatimes.com/topic/pra...,economictimes.indiatimes.com,HOLAXD,HOLAXD
3,Infosys: Will the strong volume growth sustain?,neutral,https://m.economictimes.com/markets/stocks/new...,m.economictimes.com,Infosys: Will the strong volume growth sustain?,MUMBAI: A stronger business volume growth and ...
4,Hudco raises Rs 279 cr via tax-free bonds,positive,https://economictimes.indiatimes.com/markets/b...,economictimes.indiatimes.com,Hudco raises Rs 279 cr via tax-free bonds,NEW DELHI: State-owned Housing and Urban Devel...


In [17]:
import os
# if file does not exist write header 
if not os.path.isfile('sentfin_extended.csv'):
   df.to_csv('sentfin_extended.csv', header='column_names', index=False)
else: # else it exists so append without writing the header
   df.to_csv('sentfin_extended.csv', mode='a', header=False, index=False)

# df.to_csv("file.csv", encoding='utf-8', index=False)